In [5]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [63]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [64]:
X_train = train_data.drop(columns=['label'])
y_train = train_data['label']

In [65]:
X_train = X_train / 255.0
test_data = test_data / 255.0

In [66]:
X_train = X_train.values.reshape(-1,28,28,1)
test_data = test_data.values.reshape(-1, 28, 28, 1)

In [67]:
from sklearn.model_selection import train_test_split

In [68]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1)

In [69]:
print(X_train.shape)
print(y_train.shape)

(37800, 28, 28, 1)
(37800,)


In [73]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])

optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
1182/1182 [==============================] - 58s 49ms/step - loss: 0.2844 - accuracy: 0.9112 - val_loss: 0.0819 - val_accuracy: 0.9790
Epoch 2/10
1182/1182 [==============================] - 57s 49ms/step - loss: 0.1164 - accuracy: 0.9661 - val_loss: 0.0667 - val_accuracy: 0.9795
Epoch 3/10
1182/1182 [==============================] - 57s 49ms/step - loss: 0.0950 - accuracy: 0.9739 - val_loss: 0.0465 - val_accuracy: 0.9855
Epoch 4/10
1182/1182 [==============================] - 58s 49ms/step - loss: 0.0858 - accuracy: 0.9762 - val_loss: 0.0440 - val_accuracy: 0.9864
Epoch 5/10
1182/1182 [==============================] - 58s 49ms/step - loss: 0.0882 - accuracy: 0.9754 - val_loss: 0.0395 - val_accuracy: 0.9867
Epoch 6/10
1182/1182 [==============================] - 56s 48ms/step - loss: 0.0925 - accuracy: 0.9757 - val_loss: 0.0426 - val_accuracy: 0.9864
Epoch 7/10
1182/1182 [==============================] - 57s 48ms/step - loss: 0.0881 - accuracy: 0.9753 - val_loss: 0.0527 -

In [75]:
model.save("mnist_cnn.h5")
model.evaluate(X_valid, y_valid)

132/132 [==============================] - 1s 7ms/step - loss: 0.0505 - accuracy: 0.9845


[0.05047278106212616, 0.9845238327980042]

In [76]:
predict = model.predict(test_data)

In [85]:
predict_num = []
for val in predict:
    predict_num.append(np.argmax(val))

In [87]:
submit_pd = pd.DataFrame({"ImageId": list(range(1,len(predict)+1)),
    "Label": predict_num})
submit_pd.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [88]:
submit_pd.to_csv('./Data/cnn.csv', index=False)